In [1]:
import pandas as pd
import numpy as np
import os
import datetime
os.getcwd()

# Note email cpm updated based on the Q4 client number
# Flipp hosted not included in wide version

'/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2018Q4'

In [2]:
TMR_up_to_2018Q3=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2018Q3/output/BL_MMM_long_JL_2019-01-04.csv",dtype=str)
TMR_up_to_2018Q3['impression']=TMR_up_to_2018Q3['impression'].astype(float).astype(int)
TMR_up_to_2018Q3['click']=TMR_up_to_2018Q3['click'].astype(float).astype(int)
TMR_up_to_2018Q3['cost']=TMR_up_to_2018Q3['cost'].astype(float)



In [6]:
TMR_up_to_2018Q3[(TMR_up_to_2018Q3['submedia']=="Flipp") &(TMR_up_to_2018Q3['placement']=="Native Network")]['week date'].max()

'2017-04-02'

In [7]:
TMR_up_to_2018Q3[(TMR_up_to_2018Q3['submedia']=="Flipp")]['placement'].unique()

array(['Flipp App', 'Hosted', 'Native Network'], dtype=object)

In [3]:
TMR_up_to_2018Q3_non_Email=TMR_up_to_2018Q3[TMR_up_to_2018Q3['media']!="Email"]

df_email=TMR_up_to_2018Q3[TMR_up_to_2018Q3['media']=="Email"]
df_email['cost']=df_email['impression']*0.000455784518529597

TMR_up_to_2018Q3=TMR_up_to_2018Q3_non_Email.append(df_email)

TMR_up_to_2018Q3=TMR_up_to_2018Q3.sort_values(['week date','media','submedia','placement'])
TMR_up_to_2018Q3.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2018Q3/output/BL_MMM_long_JL_2019-04-29.csv",index=False)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [4]:
TMR_up_to_2018Q3_no_DMA=TMR_up_to_2018Q3.groupby(["week date","media","submedia","placement"])['impression','click','cost'].sum().reset_index()
print(TMR_up_to_2018Q3_no_DMA['week date'].min())
print(TMR_up_to_2018Q3_no_DMA['week date'].max())

2016-09-25
2018-10-28


# TV

In [5]:
TV_logs=pd.ExcelFile("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2018Q4/Q4_TV_logs/Q4 National TMR Report 3.20.19.xlsx")
print(TV_logs.sheet_names)
TV_logs=TV_logs.parse("TMR Report 3.20.19",dtype=str,skiprows=1)


['TMR Report 3.20.19']


In [6]:
TV_logs=TV_logs[["Air Date","Vendor Net",'Act Impression','Demographic','Network']]
print(TV_logs['Demographic'].unique())
TV_logs=TV_logs[TV_logs['Demographic']=="W25-54"]
TV_logs=TV_logs[TV_logs['Air Date']!="nan"]
print(TV_logs['Demographic'].unique())

TV_logs['Vendor Net']=TV_logs['Vendor Net'].astype(float)
TV_logs['Act Impression']=TV_logs['Act Impression'].astype(int)
TV_logs=TV_logs.rename(columns={"Vendor Net":"cost","Act Impression":"impression"})

TV_logs['date']=TV_logs['Air Date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date())



['W25-54' 'HH']
['W25-54']


In [7]:
def get_week_end_date_BL(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-x.weekday())
    return y

In [8]:
TV_logs['week_end_dt']=TV_logs['date'].apply(lambda x: get_week_end_date_BL(x))
TV_logs['week date']=TV_logs['week_end_dt'].apply(lambda x: x-datetime.timedelta(days=6))
TV_TMR_data=TV_logs.groupby(['week date','Network'])[['cost','impression']].sum().reset_index()
TV_TMR_data=TV_TMR_data.rename(columns={"Network":"placement"})
TV_TMR_data['media']="TV"
TV_TMR_data['submedia']="National"
TV_TMR_data['click']=0
TV_TMR_data=TV_TMR_data[TMR_up_to_2018Q3_no_DMA.columns.tolist()]
TV_TMR_data['week date']=TV_TMR_data['week date'].astype(str)

In [9]:
print(TV_TMR_data['week date'].min())
print(TV_TMR_data['week date'].max())

2018-10-28
2018-12-23


# Others

In [10]:
Others_TMR_week=pd.read_excel("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2018Q4/data_from_Joann/DatoramaData0328.xlsx",dtype=str,sheetname="2018 Q4 TMR")
del Others_TMR_week['cleaned dma']

Others_TMR_week['week date']=Others_TMR_week['week date'].apply(lambda x: x[:10])
Others_TMR_week['impression']=Others_TMR_week['impression'].replace("nan",0).astype(float).astype(int)
Others_TMR_week['click']=Others_TMR_week['click'].replace("nan",0).astype(float).astype(int)
Others_TMR_week['cost']=Others_TMR_week['cost'].replace("nan",0).astype(float)


In [11]:
Others_TMR_week_1=Others_TMR_week[Others_TMR_week['media']!="Email"]
Others_TMR_week_2=Others_TMR_week[Others_TMR_week['media']=="Email"]

In [12]:
email_cost_client=634791
email_cost_impression=2117203832
new_email_c_p_impr=634791/2117203832

Others_TMR_week_2['cost']=Others_TMR_week_2['impression']*new_email_c_p_impr
Others_TMR_week=Others_TMR_week_1.append(Others_TMR_week_2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [13]:
new_email_c_p_impr*1000

0.2998251705412557

In [14]:
Others_TMR_week[Others_TMR_week['media']=="Circulation"]

,week date,media,submedia,placement,impression,click,cost
0,2018-11-04,Circulation,xx,xx,14415897,0,871371.0
1,2018-11-11,Circulation,xx,xx,14661742,0,871371.0
2,2018-11-18,Circulation,xx,xx,62429393,0,4199072.0
3,2018-11-25,Circulation,xx,xx,20611213,0,1230471.0
4,2018-12-02,Circulation,xx,xx,15400000,0,1085123.0
5,2018-12-09,Circulation,xx,xx,15400000,0,1085123.0
6,2018-12-16,Circulation,xx,xx,15400000,0,1085123.0
7,2019-01-06,Circulation,xx,xx,15400000,0,699502.0


In [15]:
print(Others_TMR_week_2['week date'].min())
print(Others_TMR_week_2['week date'].max())

2018-11-04
2019-01-27


# Cumulative Q4 TMR

In [16]:
print("Q3 TV last day:", TMR_up_to_2018Q3[(TMR_up_to_2018Q3['media']=="TV")]['week date'].max())

# no TV for the 2018 Oct up to Q3

Q3 TV last day: 2018-09-02


In [17]:
Q4_2018_only_TMR=Others_TMR_week.append(TV_TMR_data)

Cum_Q4_2018_TMR=TMR_up_to_2018Q3.append(Q4_2018_only_TMR)

In [18]:
Q4_2018_only_TMR.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2018Q4/BL_MMM_Q4_Only_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
Cum_Q4_2018_TMR.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2018Q4/BL_MMM_long_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [19]:
Cum_Q4_2018_TMR[(Cum_Q4_2018_TMR['submedia']=="Flipp")].head(5)

,cleaned dma,click,cost,impression,media,placement,submedia,week date
321716,"New York, NY",1941,562.89,4037,Digital,Flipp App,Flipp,2016-10-02
322291,"Dallas-Ft. Worth, TX",1799,521.71,4017,Digital,Flipp App,Flipp,2016-10-02
323963,"Los Angeles, CA",1521,441.09,2796,Digital,Flipp App,Flipp,2016-10-02
325159,"Houston, TX",1405,407.45,3267,Digital,Flipp App,Flipp,2016-10-02
325336,"Phoenix, AZ",1381,400.49,3460,Digital,Flipp App,Flipp,2016-10-02


In [20]:
Cum_Q4_2018_TMR[(Cum_Q4_2018_TMR['submedia']=="Flipp") & (Cum_Q4_2018_TMR['placement']=="Hosted")]['cost'].sum()

0.0

In [21]:
Cum_Q4_2018_TMR[Cum_Q4_2018_TMR['media']=="Circulation"]['submedia'].unique()

array(['FSI+Instore', 'Valassis', 'xx'], dtype=object)

# Wide

In [22]:
# seperate the hosted flipp
Cum_Q4_2018_TMR_FlippHost=Cum_Q4_2018_TMR[(Cum_Q4_2018_TMR['submedia']=="Flipp") & (Cum_Q4_2018_TMR['placement']=="Hosted")]
Cum_Q4_2018_TMR_FlippHost['submedia']="Flipp(hosted)"

Cum_Q4_2018_TMR_FlippNonHost=Cum_Q4_2018_TMR[(Cum_Q4_2018_TMR['submedia']=="Flipp") & (Cum_Q4_2018_TMR['placement']!="Hosted")]
Cum_Q4_2018_TMR_FlippNonHost['submedia']="Flipp(nonhosted)"


Cum_Q4_2018_TMR=Cum_Q4_2018_TMR[(Cum_Q4_2018_TMR['submedia']!="Flipp")]
Cum_Q4_2018_TMR['submedia']=np.where(Cum_Q4_2018_TMR['media']=="Circulation","xx",Cum_Q4_2018_TMR['submedia'])


Cum_Q4_2018_TMR=Cum_Q4_2018_TMR.append(Cum_Q4_2018_TMR_FlippHost).append(Cum_Q4_2018_TMR_FlippNonHost)


########################

data_media_national=Cum_Q4_2018_TMR.groupby(['week date','media'])['impression','click','cost'].sum().reset_index()
data_submedia_national=Cum_Q4_2018_TMR.groupby(['week date','media','submedia'])['impression','click','cost'].sum().reset_index()
data_submedia_national['media_submedia']=data_submedia_national['media']+"-"+data_submedia_national['submedia']
data_submedia_national=data_submedia_national[['week date','media_submedia','impression','click','cost']]


data_media_national_wide=data_media_national[['week date']].drop_duplicates()
for col in data_media_national.columns.tolist()[-3:]:
    df=data_media_national[['week date','media']+[col]].pivot(index='week date',columns='media',values=col).reset_index()
    df_new_col_list=df.columns.tolist()
    df_new_col_list.remove('week date')
    df_new_col_dict={}
    for new_col in df_new_col_list:
        df_new_col_dict.update({new_col:new_col+"_"+col})
    df=df.rename(columns=df_new_col_dict)
    data_media_national_wide=pd.merge(data_media_national_wide,df,on='week date',how="outer")
data_media_national_wide=data_media_national_wide.fillna(0)
data_media_national_wide=data_media_national_wide.drop_duplicates()
#############

data_submedia_national_wide=data_submedia_national[['week date']].drop_duplicates()
for col in data_submedia_national.columns.tolist()[-3:]:
    df=data_submedia_national[['week date','media_submedia']+[col]].pivot(index='week date',columns='media_submedia',values=col).reset_index()
    df_new_col_list=df.columns.tolist()
    df_new_col_list.remove('week date')
    df_new_col_dict={}
    for new_col in df_new_col_list:
        df_new_col_dict.update({new_col:new_col+"_"+col})
    df=df.rename(columns=df_new_col_dict)
    data_submedia_national_wide=pd.merge(data_submedia_national_wide,df,on='week date',how="outer")
data_submedia_national_wide=data_submedia_national_wide.fillna(0)
data_submedia_national_wide=data_submedia_national_wide.drop_duplicates()


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
including_weeks=[str(datetime.date(2019,2,2)-datetime.timedelta(days=7*x+6)) for x in range(104)]


data_media_national_wide=data_media_national_wide.sort_values("week date")
data_media_national_wide=data_media_national_wide[data_media_national_wide['week date'].isin(including_weeks)]

data_submedia_national_wide=data_submedia_national_wide.sort_values("week date")
data_submedia_national_wide=data_submedia_national_wide[data_submedia_national_wide['week date'].isin(including_weeks)]



## Sales

In [24]:
sales_data=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-02-09.csv",dtype=str)
sales_data['week_end_date']=sales_data['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
sales_data['sales']=sales_data['sales'].astype(float)
sales_data['transactions']=sales_data['transactions'].astype(float)
sales_data['week date']=sales_data['week_end_date'].apply(lambda x: x-datetime.timedelta(days=6))
sales_data_naitonal=sales_data.groupby(['week date'])['sales','transactions'].sum().reset_index()
sales_data_naitonal=sales_data_naitonal.set_index(['week date'])
sales_data_naitonal.at[datetime.date(2017,4,30),'transactions']=(sales_data_naitonal.at[datetime.date(2017,4,23),'transactions']+sales_data_naitonal.at[datetime.date(2017,5,7),'transactions'])/2
sales_data_naitonal=sales_data_naitonal.reset_index()
sales_data_store_counts=sales_data[sales_data['sales']>0]
sales_data_store_counts=sales_data_store_counts.groupby(['week date'])['location_id'].count().to_frame().reset_index().rename(columns={"location_id":"store_counts"})

sales_data_naitonal=pd.merge(sales_data_naitonal,sales_data_store_counts,on="week date",how="outer")
sales_data_naitonal['week date']=sales_data_naitonal['week date'].astype(str)

In [25]:
data_media_national_wide=pd.merge(data_media_national_wide,sales_data_naitonal,on="week date",how="left")
data_submedia_national_wide=pd.merge(data_submedia_national_wide,sales_data_naitonal,on="week date",how="left")

## Promotion

In [26]:
Rewards_Promotion_list=[datetime.date(2016,10,1),datetime.date(2016,10,2),
                        datetime.date(2017,1,21),datetime.date(2017,1,22),
                        datetime.date(2017,4,1),datetime.date(2017,4,2),
                        datetime.date(2017,7,8),datetime.date(2017,7,9),
                        datetime.date(2017,9,30),datetime.date(2017,10,1),
                        datetime.date(2018,1,20),datetime.date(2018,1,21),
                        datetime.date(2018,4,7),datetime.date(2018,4,8),
                        datetime.date(2018,7,7),datetime.date(2018,7,8),
                        datetime.date(2018,9,29),datetime.date(2018,9,30),
                        datetime.date(2019,1,19),datetime.date(2019,1,20)]

df_Rewards_Promotion=pd.DataFrame({"Date":Rewards_Promotion_list},index=range(len(Rewards_Promotion_list)))
df_Rewards_Promotion['weekday']=df_Rewards_Promotion['Date'].apply(lambda x: x.weekday())
df_Rewards_Promotion['week date']=np.where(df_Rewards_Promotion['weekday']==6,df_Rewards_Promotion['Date'],df_Rewards_Promotion['Date']-datetime.timedelta(days=6))

del df_Rewards_Promotion['Date']

df_Rewards_Promotion_Sunday=df_Rewards_Promotion[df_Rewards_Promotion['weekday']==6]
df_Rewards_Promotion_Sunday['Sunday_rewards_ind']=1
del df_Rewards_Promotion_Sunday['weekday']
df_Rewards_Promotion_Sunday['week date']=df_Rewards_Promotion_Sunday['week date'].astype(str)


df_Rewards_Promotion_Saturday=df_Rewards_Promotion[df_Rewards_Promotion['weekday']==5]
df_Rewards_Promotion_Saturday['Saturday_rewards_ind']=1
del df_Rewards_Promotion_Saturday['weekday']
df_Rewards_Promotion_Saturday['week date']=df_Rewards_Promotion_Saturday['week date'].astype(str)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [27]:
holiday_1_week_df=pd.DataFrame({"week date":[datetime.date(2016,12,18),datetime.date(2017,12,17),datetime.date(2018,12,23)],"Holiday_1_week_only_Ind":[1]*3},index=[0,1,2])
holiday_5_weeks_df=pd.DataFrame({"week date":[datetime.date(2016,12,18)-datetime.timedelta(days=x*7) for x in range(5)] +[datetime.date(2017,12,17)-datetime.timedelta(days=x*7) for x in range(5)]+[datetime.date(2018,12,23)-datetime.timedelta(days=x*7) for x in range(5)],
                                 "Holiday_5_weeks_only_Ind":[1]*15},index=[x for x in range(15)])

holiday_1_week_df['week date']=holiday_1_week_df['week date'].astype(str)
holiday_5_weeks_df['week date']=holiday_5_weeks_df['week date'].astype(str)


df_special_weeks_ind=pd.merge(df_Rewards_Promotion_Saturday,df_Rewards_Promotion_Sunday,on="week date",how="outer")
df_special_weeks_ind=pd.merge(df_special_weeks_ind,holiday_1_week_df,on="week date",how="outer")
df_special_weeks_ind=pd.merge(df_special_weeks_ind,holiday_5_weeks_df,on="week date",how="outer")


In [28]:
data_media_national_wide=pd.merge(data_media_national_wide,df_special_weeks_ind,on="week date",how="left")

data_submedia_national_wide=pd.merge(data_submedia_national_wide,df_special_weeks_ind,on="week date",how="left")

data_media_national_wide=data_media_national_wide.fillna(0)
data_submedia_national_wide=data_submedia_national_wide.fillna(0)

In [29]:
data_media_national_wide.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2018Q4/output_wide/BL_MMM_data_media_national_wide_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
data_submedia_national_wide.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2018Q4/output_wide/BL_MMM_data_submedia_national_wide_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

# Note email cpm updated based on the Q4 client number
# Flipp hosted seperated in wide version